# Data collection
Update COVID case and death data from: https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pylab as plt
%matplotlib inline

import seaborn as sns

## SES data

In [2]:
pe = pd.read_csv('../data/PovertyEstimates.csv', thousands=',').rename(columns={'FIPStxt': 'fips'})
pe['fips'] = pe['fips'].apply(lambda x: str(x).zfill(5))
pe = pe.set_index('fips')[['POVALL_2018', 'PCTPOVALL_2018', 'MEDHHINC_2018']]
pe.shape

(3193, 3)

## Rural data

In [3]:
df_rural = pd.read_csv('../data/County_Rural_Lookup.csv', thousands=',', usecols=[0, 7])
df_rural = df_rural.rename(columns={'2015 GEOID': 'fips', '2010 Census \nPercent Rural': 'perc_rural_pop'})
df_rural['perc_urban_pop'] = 100 - df_rural['perc_rural_pop']
df_rural = df_rural.set_index('fips').dropna()
print(df_rural.shape)

(3142, 2)


## Demographic data

In [4]:
dm_raw = pd.read_csv('../data/cc-est2019-alldata.csv', encoding='ISO-8859-1', dtype={'STATE': str, 'COUNTY': str})
dm_raw = dm_raw[dm_raw.YEAR==12]
dm_raw['fips'] = dm_raw.STATE + dm_raw.COUNTY
dm_raw = dm_raw.set_index('fips')
dm_raw.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
fips,,,,,,,,,,,,,,,,,,,,,
01001,50,01,001,Alabama,Autauga County,12,0,55869,27092,28777,...,778,687,89,93,40,27,15,19,16,11
01001,50,01,001,Alabama,Autauga County,12,1,3277,1713,1564,...,76,53,10,6,6,5,3,4,3,3
01001,50,01,001,Alabama,Autauga County,12,2,3465,1787,1678,...,83,59,2,10,8,2,2,0,1,1
01001,50,01,001,Alabama,Autauga County,12,3,3851,1977,1874,...,84,67,11,12,2,2,1,2,2,1
01001,50,01,001,Alabama,Autauga County,12,4,3659,1854,1805,...,55,68,7,6,4,5,0,4,3,0


In [5]:
dm_raw['minority'] = dm_raw.TOT_POP - (dm_raw.WA_MALE + dm_raw.WA_FEMALE)
dm_raw['black'] = dm_raw.BA_MALE + dm_raw.BA_FEMALE
dm_raw['hispanic'] = dm_raw.H_MALE + dm_raw.H_FEMALE
dm_all = dm_raw[dm_raw.AGEGRP == 0][['minority', 'black', 'hispanic', 'TOT_POP']]

In [6]:
dm_old = dm_raw[['AGEGRP', 'TOT_POP']][dm_raw.AGEGRP >= 14].reset_index().groupby('fips').sum()
dm_old = dm_old.rename(columns={'TOT_POP': '65yrs'}).drop(['AGEGRP'], axis=1)

In [7]:
dm = dm_old.join(dm_all)

In [8]:
columns = dm.columns
for c in ['65yrs', 'minority', 'black', 'hispanic']:
    pc = 'perc_' + c
    dm[pc] = dm[c] / dm.TOT_POP

### Sanity check for demographic dataframe (dm)
passed

In [9]:
summation = dm.sum(axis=0)
perc_black = summation['black'] / summation['TOT_POP']
print(f'black percentage = {100 * perc_black:.1f}%')
perc_hispanic = summation['hispanic'] / summation['TOT_POP']
print(f'hispanic percentage = {100 * perc_hispanic:.1f}%')
perc_minority = summation['minority'] / summation['TOT_POP']
print(f'minority percentage = {100 * perc_minority:.1f}%')
print(f"population over 65yrs = {summation['65yrs']}")

black percentage = 13.4%
hispanic percentage = 18.5%
minority percentage = 23.7%
population over 65yrs = 54058263.0


# Covid data

In [49]:
start = '4/4/20'
step = 7

## Covid case data

In [59]:
df_covid_case_raw = pd.read_csv('../data/time_series_covid19_confirmed_US_2020-11-03.csv').dropna()
df_covid_case_raw['FIPS'] = df_covid_case_raw['FIPS'].apply(lambda x: str(int(x)).zfill(5))
df_covid_case_raw = df_covid_case_raw.rename(columns={'FIPS': 'fips'}).set_index('fips')
df_covid_case_raw = df_covid_case_raw.drop([
    'UID', 'iso2', 'iso3', 'code3', 
    'Admin2', 'Province_State', 'Country_Region', 
    'Lat', 'Long_', 'Combined_Key'], axis=1)

df_covid_case_raw['1/21/20'] = 0

### Get step sub-dataset

In [60]:
columns = df_covid_case_raw.columns.values
start_idx = np.argwhere(columns==start)[0][0]

selected = columns[np.arange(start_idx, len(columns), step)]

foo = lambda x: '-'.join(x.split('/')[:2])

df_covid_case_step = df_covid_case_raw[selected].rename(columns={col: foo(col) + '_c' for col in selected})

selected_ = [foo(col) for col in selected]
with open('steps.dat', 'w') as handle:
    handle.write(' '.join(selected_))

### get period prevalence

In [61]:
df_covid_case = df_covid_case_step[[foo(selected[0]) + '_c']]
df_covid_case = df_covid_case.join(df_covid_case_step.diff(axis=1)[df_covid_case_step.columns[1:]])

## Covid death data

In [62]:
df_covid_death_raw = pd.read_csv('../data/time_series_covid19_death_US_2020-11-02.csv').dropna()
df_covid_death_raw['FIPS'] = df_covid_death_raw['FIPS'].apply(lambda x: str(int(x)).zfill(5))
df_covid_death_raw = df_covid_death_raw.rename(columns={'FIPS': 'fips'}).set_index('fips')
df_covid_death_raw = df_covid_death_raw.drop([
    'UID', 'iso2', 'iso3', 'code3', 
    'Admin2', 'Province_State', 'Country_Region', 
    'Lat', 'Long_', 'Combined_Key'], axis=1)

df_covid_death_raw['1/21/20'] = 0

### Get step sub-dataset

In [63]:
columns = df_covid_death_raw.columns.values
start_idx = np.argwhere(columns==start)[0][0]

selected = columns[np.arange(start_idx, len(columns), step)]

foo = lambda x: '-'.join(x.split('/')[:2])

df_covid_death_step = df_covid_death_raw[selected].rename(columns={col: foo(col) + '_d'  for col in selected})

selected_ = [foo(col) for col in selected]
with open('steps.dat', 'w') as handle:
    handle.write(' '.join(selected_))

### get period prevalence

In [64]:
df_covid_death = df_covid_death_step[[foo(selected[0]) + '_d']]
df_covid_death = df_covid_death.join(df_covid_death_step.diff(axis=1)[df_covid_death_step.columns[1:]])

## Combine

In [65]:
df_covid = df_covid_case.join(df_covid_death)

## Risk data

In [66]:
rf = pd.read_csv('../data/county_pop_risk_covid.csv', dtype={'county': str})
rf = rf.rename(columns={'county': 'fips'}).set_index('fips')
rf.fillna(rf.mean(), inplace=True)
print(rf.shape)

(3094, 15)


## Combine demographic, SES, rural, risk, and covid dataframe

In [67]:
risk_cols = ['risk', 'risk_flu']
df = dm.join(pe, how='inner')\
    .join(rf[risk_cols], how='inner')\
    .join(df_rural, how='inner')\
    .join(df_covid, how='inner')\
    .rename(columns={
        'POVALL_2018': 'poverty',
        'PCTPOVALL_2018': 'perc_poverty', 
        'MEDHHINC_2018': 'income', 
        'TOT_POP': 'population'})
print(df.columns)

Index(['65yrs', 'minority', 'black', 'hispanic', 'population', 'perc_65yrs',
       'perc_minority', 'perc_black', 'perc_hispanic', 'poverty',
       'perc_poverty', 'income', 'risk', 'risk_flu', 'perc_rural_pop',
       'perc_urban_pop', '4-4_c', '4-11_c', '4-18_c', '4-25_c', '5-2_c',
       '5-9_c', '5-16_c', '5-23_c', '5-30_c', '6-6_c', '6-13_c', '6-20_c',
       '6-27_c', '7-4_c', '7-11_c', '7-18_c', '7-25_c', '8-1_c', '8-8_c',
       '8-15_c', '8-22_c', '8-29_c', '9-5_c', '9-12_c', '9-19_c', '9-26_c',
       '10-3_c', '10-10_c', '10-17_c', '10-24_c', '10-31_c', '4-4_d', '4-11_d',
       '4-18_d', '4-25_d', '5-2_d', '5-9_d', '5-16_d', '5-23_d', '5-30_d',
       '6-6_d', '6-13_d', '6-20_d', '6-27_d', '7-4_d', '7-11_d', '7-18_d',
       '7-25_d', '8-1_d', '8-8_d', '8-15_d', '8-22_d', '8-29_d', '9-5_d',
       '9-12_d', '9-19_d', '9-26_d', '10-3_d', '10-10_d', '10-17_d', '10-24_d',
       '10-31_d'],
      dtype='object')


In [68]:
print(df.shape)
df.to_csv(f'combined_data_step-{step}.csv', float_format='%.6f')

(3094, 78)
